**Cross-Encoder Re-Ranking**

Install sentence-transformers python library  and import 'CrossEncoder' from this library.

In [ ]:
# place your code here


In [ ]:
# place your code here

From typing  import List, Tuple

In [ ]:
# place your code here

Load the pre-trained cross-encoder model 'cross-encoder/ms-marco-MiniLM-L-6-v2' and save the model object to the 'model' variable. To load, use CrossEncoder() with two arguments: the first is the model name as string and the second argument is 'max_length=512'

In [ ]:
# place your code here

Use function:

In [ ]:
def rerank_documents(
    query: str,  documents: List[str],  top_k: int = 10
) -> List[Tuple[int, str, float]]:
    """
    Re-rank documents based on relevance to query.
    Args:
        query: The search query
        documents: List of document texts from initial retrieval
        top_k: Number of top results to return
    Returns:
        List of (original_index, document, score) tuples, sorted by relevance
    """
    # Create query-document pairs for the cross-encoder
    # Each pair will be scored independently
    pairs = [[query, doc] for doc in documents]

    # Get relevance scores for all pairs
    # The model outputs a single score per pair
    scores = model.predict(pairs)

    # Combine with original indices and sort by score descending
    scored_docs = [
        (idx, doc, float(score))
        for idx, (doc, score) in enumerate(zip(documents, scores))
    ]
    scored_docs.sort(key=lambda x: x[2], reverse=True)

    return scored_docs[:top_k]

Simulate retrieval results: use the following data.

In [ ]:
retrieved_docs = [
        "Kubernetes pod monitoring requires metrics collection from the kubelet.",
        "Docker containers can be monitored using cAdvisor metrics.",
        "To check pod status, use kubectl get pods command.",
        "Prometheus is commonly used for Kubernetes monitoring.",
        "Pod resource limits should be set in the deployment spec.",
        "OneUptime provides real-time Kubernetes monitoring dashboards.",
        "Container orchestration platforms need observability solutions.",
        "The kubectl logs command shows container output.",
    ]
query = "How do I monitor Kubernetes pods?"

Select a value for the k parameter and apply the "rerank_documents()" function to the data.
Save the returned object to the results variabl.

In [ ]:
# place your code here.

Using the 'for idx, doc, score in results'    loop, print the values ​​of 'idx', 'doc' and 'score'

In [ ]:
print("Re-ranked results:")
# place your code here